In [1]:
import pandas as pd
data = pd.read_csv('./Data/20240205fullsample_new.csv')
data.head()

,group,id,loan_id,EID,installment,action,action_num_should,action_num_actual,cumu_action0,cumu_action1,...,reward_recovery_rate_nn13,reward_recovery_rate_nn14,delay_due_days,delay_due_days_diff,recovery_rate_total,loan_done,installment_timestep,recovery_rate_weights,recovery_rate_weighted,sample
0,train,51,/+6C2lDDYJgzzCXpn96AFA==,4,1,no_action,0,0,0,0,...,0.033353,0.034837,0,0,0.000000,0,1,1.062157,0.205350,rlsimulator
1,train,53,/+6C2lDDYJgzzCXpn96AFA==,4,2,self,1,1,1,0,...,0.052457,0.031081,2,2,0.000000,0,1,0.868746,0.000000,rlsimulator
2,train,54,/+6C2lDDYJgzzCXpn96AFA==,4,2,family,2,2,1,1,...,0.052457,0.019864,7,5,0.000000,0,2,0.868746,0.000000,rlsimulator
3,train,55,/+6C2lDDYJgzzCXpn96AFA==,4,2,acquiantance,3,3,1,1,...,0.088502,0.045893,7,0,0.000000,0,3,0.868746,0.000000,rlsimulator
4,train,56,/+6C2lDDYJgzzCXpn96AFA==,4,2,sms,4,4,1,1,...,0.088502,0.097007,8,1,0.185833,0,4,0.868746,0.177079,rlsimulator


The purpose of `installment_timestep` is to generate a timestep for each installment of each loan (loan id), indicating the order in which that installment is repaid in the current loan.

In [22]:
data['installment_timestep'] = data.groupby(
    ['loan_id', 'installment']).cumcount() + 1

In [31]:
data_sim = data.loc[data['sample'] == 'rlsimulator']
data_sim

,group,id,loan_id,EID,installment,action,action_num_should,action_num_actual,cumu_action0,cumu_action1,...,reward_recovery_rate_nn13,reward_recovery_rate_nn14,delay_due_days,delay_due_days_diff,recovery_rate_total,loan_done,installment_timestep,recovery_rate_weights,recovery_rate_weighted,sample
0,train,51,/+6C2lDDYJgzzCXpn96AFA==,4,1,no_action,0,0,0,0,...,0.033353,0.034837,0,0,0.000000,0,1,1.062157,0.205350,rlsimulator
1,train,53,/+6C2lDDYJgzzCXpn96AFA==,4,2,self,1,1,1,0,...,0.052457,0.031081,2,2,0.000000,0,1,0.868746,0.000000,rlsimulator
2,train,54,/+6C2lDDYJgzzCXpn96AFA==,4,2,family,2,2,1,1,...,0.052457,0.019864,7,5,0.000000,0,2,0.868746,0.000000,rlsimulator
3,train,55,/+6C2lDDYJgzzCXpn96AFA==,4,2,acquiantance,3,3,1,1,...,0.088502,0.045893,7,0,0.000000,0,3,0.868746,0.000000,rlsimulator
4,train,56,/+6C2lDDYJgzzCXpn96AFA==,4,2,sms,4,4,1,1,...,0.088502,0.097007,8,1,0.185833,0,4,0.868746,0.177079,rlsimulator
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200009,test,63266,ZWocy2KIKqJbgKZVoXZIFA==,59,4,sms,4,0,4,4,...,NaN,NaN,15,3,0.000000,0,4,0.109583,0.000000,rlsimulator
200010,test,63268,ZWocy2KIKqJbgKZVoXZIFA==,59,5,self,1,1,5,4,...,0.031653,0.045248,2,2,0.000000,0,1,0.148145,0.000000,rlsimulator
200011,test,63269,ZWocy2KIKqJbgKZVoXZIFA==,59,5,family,2,0,5,5,...,NaN,NaN,5,3,0.000000,0,2,0.148145,0.000000,rlsimulator
200012,test,63270,ZWocy2KIKqJbgKZVoXZIFA==,59,5,acquiantance,3,0,6,5,...,NaN,NaN,8,3,0.000000,0,3,0.148145,0.000000,rlsimulator


In [38]:
loan_id_list = data_sim['loan_id'].unique().tolist()
# len(loan_id_list)
loan_id_list

['/+6C2lDDYJgzzCXpn96AFA==',
 '/+CZ3xwX6Y7HF0jKRJatPQ==',
 '/00aMf+3VqliBoTmDIWyOw==',
 '/0BQUoMdcCXhRI0AJqPjIQ==',
 '/0TKh6YVH7lNO/g0E58Y+A==',
 '/1EeT8dshjzYTvW0mQwHuA==',
 '/1zsCT1Y4csYWXbmDMB1xA==',
 '/2WOuLrj0rVAxY78wVB7+w==',
 '/3JkKgbGSk9wa9TpEOfoHg==',
 '/3OaIjsOiopXeRRXrFwhhA==',
 '/4+vGQ3fi7qurO0drsm/wA==',
 '/4kCDxckZgvKgUscmfgOCg==',
 '/4Z/h3eEYTE4gWCJzdMT+g==',
 '/6+uarg7ytBc5xz2sKjW4g==',
 '/6i6/w4JWvrQ4hihtbeebA==',
 '/7rxGgM49G3NNwhyvRDspQ==',
 '/81RN1NTlBjBUFscs0ndSA==',
 '/8fY+oY03kDJ93qyEX3v4w==',
 '/8G193xaSSpMViyXTrH6yA==',
 '/8nOCSSp4g1s4ShCUQGXow==',
 '/8qN303VftUERSTC9rXQIw==',
 '/8xqjGY6qnfUvXKagT5hTA==',
 '/9f6njYsIFQLFZNCrpCaag==',
 '/9UXH1bpvmnGK9ZT9fp+Qw==',
 '/A/zumCR33mpFIRg92DqeQ==',
 '/A5jf8yo6y9u/4L5uFZEvA==',
 '/AADuL2y/KExtnXKhaZfpg==',
 '/aSp0oSyhUrgxnlM0vjm1w==',
 '/AXB3KuE2xli0HLxmAhrBg==',
 '/BjmOrzdtWJ0oWR2fM56hw==',
 '/C1nXWYUf8pGLZHHtB5w0Q==',
 '/CAOi5rwoJNOjyuOFIgqgA==',
 '/CKHlpDIQXK8sCTOOAvQEw==',
 '/d9QA/8Afj5gljOSQT6geg==',
 '/dJ5Dz2ysBt+

In [40]:
# specific columns names
loan_id = ['loan_id']
user_features = ['gender',
                 'age',
                 'amount',
                 'num_loan',
                 'duration',
                 'year_ratio',
                 'diff_city',
                 'marriage',
                 'kids',
                 'month_in',
                 'housing',
                 'edu',
                 'motivation']
current_state = ['installment',
                 'installment_timestep',
                 'state_cum_overduelength',
                 'remaining_debt',
                 'state_capital',
                 'state_interests',
                 'state_penalty',
                 ]
other_labels = ['installment_done',
                'loan_done',
                'recovery_rate_weighted']

In [42]:
example_id = '/+6C2lDDYJgzzCXpn96AFA=='
example_data = data_sim.loc[data_sim['loan_id'] == example_id]

In [43]:
x_train = pd.DataFrame()
x_train = example_data[current_state]
x_train

,installment,installment_timestep,state_cum_overduelength,remaining_debt,state_capital,state_interests,state_penalty
0,1,1,0,2000.000000,333.333333,26.666667,0.000000
1,2,1,2,2000.000000,666.666667,53.333333,3.333333
2,2,2,7,2000.000000,666.666667,53.333333,11.666667
3,2,3,7,2000.000000,666.666667,53.333333,11.666667
4,2,4,8,2000.000000,666.666667,53.333333,13.333333
5,3,1,0,1666.666667,666.666667,53.333333,0.000000
6,3,2,0,1666.666667,666.666667,53.333333,0.000000
7,4,1,2,1333.333333,666.666667,53.333333,3.333333
8,4,2,4,1333.333333,666.666667,53.333333,6.666667
9,4,3,6,1333.333333,666.666667,53.333333,10.000000


In [48]:
y_train = pd.DataFrame()
y_train = example_data[current_state]
col_matching = {'installment': 'y_installment',
                'installment_timestep': 'y_installment_timestep',
                'state_cum_overduelength': 'y_state_cum_overduelength',
                'remaining_debt': 'y_remaining_debt',
                'state_capital': 'y_state_capital',
                'state_interests': 'y_state_interests',
                'state_penalty': 'y_state_penalty'}
y_train = y_train.rename(columns=col_matching)
if y_train.shape[0] > 1:
    y_train = y_train[1:]
    y_train = pd.concat(
        [y_train, y_train.iloc[[-1]]], ignore_index=True)

y_train

,y_installment,y_installment_timestep,y_state_cum_overduelength,y_remaining_debt,y_state_capital,y_state_interests,y_state_penalty
0,2,1,2,2000.000000,666.666667,53.333333,3.333333
1,2,2,7,2000.000000,666.666667,53.333333,11.666667
2,2,3,7,2000.000000,666.666667,53.333333,11.666667
3,2,4,8,2000.000000,666.666667,53.333333,13.333333
4,3,1,0,1666.666667,666.666667,53.333333,0.000000
5,3,2,0,1666.666667,666.666667,53.333333,0.000000
6,4,1,2,1333.333333,666.666667,53.333333,3.333333
7,4,2,4,1333.333333,666.666667,53.333333,6.666667
8,4,3,6,1333.333333,666.666667,53.333333,10.000000
9,4,4,8,1333.333333,666.666667,53.333333,13.333333
